# Model scoring
Score all sequences in generated_seq using only RBM-ACE2


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import sys
from sklearn.metrics import roc_auc_score
import matplotlib as mpl
import matplotlib.dates as mdates
from scipy.stats import ttest_rel
from sklearn.metrics import roc_auc_score
from pathlib import Path
from scipy.stats import ttest_rel
from sklearn.metrics import roc_auc_score
import re, glob
from typing import List, Tuple
import os


sys.path.append("../")
from utils import *
from global_variables import *
from escape_map import *
pgm_path = "PGM/"
sys.path.append(pgm_path + "source/")
sys.path.append(pgm_path + "utilities/")
import utilities, Proteins_utils, sequence_logo, plots_utils
import rbm, RBM_utils
from Proteins_utils import load_FASTA

kd_vec=KD_VECTORS|NEW_KD_VECTORS
kd_ace2=ACE2_KD_VECTOR



E:\ESCAPE_MAP_DRAFT\PGM\source\numba_utilities.py:1124: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float32, 2, 'F', False, aligned=True), Array(float32, 2, 'A', False, aligned=True))
  dmean_v_dw = np.dot(s1.T, V)
E:\ESCAPE_MAP_DRAFT\PGM\source\numba_utilities.py:961: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float32, 1, 'A', False, aligned=True), Array(float32, 2, 'A', False, aligned=True))
  mean_V = np.dot(weights, V) / sum_weights
C:\Users\maria\AppData\Roaming\Python\Python312\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


Loaded 29 KD vectors
Loaded 2 KD vectors
Loaded 671 KD vectors
Loaded 438 KD vectors


In [2]:
os.listdir()

['6m0j.cif',
 'esmif_format_conv.ipynb',
 'generated_seq',
 'model_scoring.ipynb',
 'plot_esmif.ipynb',
 'pymol.ipynb',
 'script_generate_seq.py',
 'seq_esmif_format',
 'seq_esmif_scores',
 'seq_scores_model',
 'top_sites_escape.png']

In [7]:

ab_names=list(kd_vec.keys())

train_date = pd.Timestamp("2020-04-01")  # or any date-like


em_path = "../params_by_period_0.03_m1_m8_m8/param_period_"+train_date.strftime("%Y-%m-%d")+".csv"
model=load_escape_map_from_csv(em_path)
model.raw_concentrations=model.raw_concentrations*0 -14



In [8]:
wt_rbd=Proteins_utils.load_FASTA('../seq_data/rbd_wt.fasta')[:,BEGIN:-END]
print(wt_rbd.shape)
wt_score=-model(wt_rbd[0])
print(wt_score)


(1, 178)
1355.1987


In [10]:
seq=Proteins_utils.load_FASTA('generated_seq/generated_seq_Beta0.4.fasta')
score=-model(seq)[0]
score

1144.6337043639116

In [11]:
input_dir = "generated_seq"
output_dir = "seq_scores_model"
os.makedirs(output_dir, exist_ok=True)

# Process all FASTA files in the input directory
for fasta_filename in os.listdir(input_dir):
    if fasta_filename.endswith(".fasta"):
        input_path = os.path.join(input_dir, fasta_filename)
        output_path = os.path.join(output_dir, f"model_lll_{fasta_filename.replace('.fasta', '.csv')}")
        
        # Load sequences from FASTA file
        seq=Proteins_utils.load_FASTA('generated_seq/'+fasta_filename)
        scores = [-model(s) for s in seq]
        seqids = [f"{i}" for i in range(len(seq))]
        # Save scores to CSV
        df = pd.DataFrame({"seqid": seqids, "model_score": scores})
        df.to_csv(output_path, index=False)
        
        print(f"Processed {fasta_filename}")

print("All FASTA files processed.")


Processed generated_seq_Beta0.4.fasta
Processed generated_seq_Beta0.6.fasta
Processed generated_seq_Beta0.8.fasta
Processed generated_seq_Beta1.0.fasta
Processed generated_seq_Beta1.0_Ab_COV2-2130.fasta
Processed generated_seq_Beta1.0_Ab_COV2-2196.fasta
Processed generated_seq_Beta1.0_Ab_LY-CoV555.fasta
Processed generated_seq_Beta1.0_Ab_REGN10933.fasta
Processed generated_seq_Beta1.0_Ab_S2E12.fasta
Processed generated_seq_Beta1.0_Ab_S309.fasta
Processed generated_seq_Beta1.0_Ab_SA55.fasta
Processed generated_seq_Beta1.0_Ab_VIR7229.fasta
Processed generated_seq_Beta1.5.fasta
Processed generated_seq_Beta2.0.fasta
Processed generated_seq_Beta3.0.fasta
All FASTA files processed.
